# Quinto notebook

En el siguiente notebook se  registrara de forma ordenada los experimentos de entrenamiento
del modelo YOLO usando MLflow.

Este notebook:
- Configura MLflow en modo local
- Registra parámetros del experimento
- Registra métricas de evaluación (mAP, precision, recall)
- Guarda artefactos importantes (modelo, data.yaml)
- Deja listo el historial para comparar futuros entrenamientos


In [ ]:
# --- IMPORTS ---
from pathlib import Path
import json
import shutil


In [ ]:
# --- RUTAS DEL PROYECTO ---
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent

MODELS_DIR = PROJECT_ROOT / "models"
RUNS_DIR = PROJECT_ROOT / "runs"
ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
MLRUNS_DIR = PROJECT_ROOT / "mlruns"

CONFIG_SNAPSHOT_PATH = ARTIFACTS_DIR / "config_snapshot.json"
CLASS_MAP_PATH = ARTIFACTS_DIR / "class_map.json"

YOLO_RUN_DIR = RUNS_DIR / "yolo_coco_subset"
BEST_MODEL_PATH = MODELS_DIR / "yolo_best.pt"

print("MLRUNS_DIR:", MLRUNS_DIR)


In [ ]:
# --- VALIDACIONES ---
def assert_exists(p: Path, desc: str) -> None:
    if not p.exists():
        raise FileNotFoundError(f"Falta {desc}: {p}")

assert_exists(BEST_MODEL_PATH, "modelo entrenado yolo_best.pt")
assert_exists(CONFIG_SNAPSHOT_PATH, "config_snapshot.json")
assert_exists(CLASS_MAP_PATH, "class_map.json")

print("Validaciones OK.")


In [ ]:
# --- CONFIGURACIÓN DE MLFLOW ---
import mlflow

MLFLOW_TRACKING_URI = f"file:{MLRUNS_DIR.as_posix()}"
MLFLOW_EXPERIMENT_NAME = "coco2017_car_airplane_truck"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

print("Tracking URI:", MLFLOW_TRACKING_URI)
print("Experiment:", MLFLOW_EXPERIMENT_NAME)


In [ ]:
# --- CARGA DE CONFIGURACIÓN ---
with open(CONFIG_SNAPSHOT_PATH, "r", encoding="utf-8") as f:
    cfg = json.load(f)

with open(CLASS_MAP_PATH, "r", encoding="utf-8") as f:
    class_map = json.load(f)

cfg, class_map


In [ ]:
# --- EXTRACCIÓN DE MÉTRICAS DESDE YOLO ---
# YOLO guarda resultados en results.csv dentro del run
RESULTS_CSV = YOLO_RUN_DIR / "results.csv"

if not RESULTS_CSV.exists():
    raise FileNotFoundError(
        "No se encontró results.csv. "
        "Asegúrate de haber ejecutado el Notebook 03."
    )

import csv

metrics = {}

with open(RESULTS_CSV, newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    rows = list(reader)
    last_row = rows[-1]

metrics["precision"] = float(last_row.get("metrics/precision(B)", 0))
metrics["recall"] = float(last_row.get("metrics/recall(B)", 0))
metrics["mAP50"] = float(last_row.get("metrics/mAP50(B)", 0))
metrics["mAP50_95"] = float(last_row.get("metrics/mAP50-95(B)", 0))

metrics


In [ ]:
# --- REGISTRO DEL EXPERIMENTO EN MLFLOW ---
with mlflow.start_run(run_name="yolo_coco_subset_limited"):

    # Parámetros
    mlflow.log_params({
        "model_type": "YOLOv8",
        "base_model": "yolov8n.pt",
        "img_size": cfg["img_size"],
        "batch_size": cfg["batch_size"],
        "epochs": cfg["epochs"],
        "optimizer": "SGD",
        "dataset_mode": "limited",
        "classes": ",".join(cfg["target_classes"])
    })

    # Métricas
    mlflow.log_metrics(metrics)

    # Artefactos
    mlflow.log_artifact(BEST_MODEL_PATH, artifact_path="model")
    mlflow.log_artifact(CONFIG_SNAPSHOT_PATH, artifact_path="config")
    mlflow.log_artifact(CLASS_MAP_PATH, artifact_path="config")

print("Experimento registrado en MLflow.")


In [ ]:
# --- REGISTRO DE ARTEFACTOS ADICIONALES (OPCIONAL) ---
if YOLO_RUN_DIR.exists():
    mlflow.log_artifacts(YOLO_RUN_DIR, artifact_path="training_outputs")
    print("Artefactos de entrenamiento guardados.")


In [ ]:
# --- RESUMEN FINAL ---
print("MLflow configurado correctamente.")
print("Modelo registrado:", BEST_MODEL_PATH)
print("Métricas registradas:")
for k, v in metrics.items():
    print("-", k, ":", v)